In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("lakshmi25npathi/imdb-dataset-of-50k-movie-reviews")

print("Path to dataset files:", path)

100%|██████████| 18.6M/18.6M [00:02<00:00, 9.68MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/shusrith/imbd-dataset-tokenized/versions/3


In [ ]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [3]:
import pandas as pd

df = pd.read_csv("/root/.cache/kagglehub/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews/versions/1/IMDB Dataset.csv")
df

,review,sentiment
0,"[101, 2028, 1997, 1996, 2060, 15814, 2038, 385...",1
1,"[101, 1037, 6919, 2210, 2537, 7987, 7987, 1996...",1
2,"[101, 1045, 2245, 2023, 2001, 1037, 6919, 2126...",1
3,"[101, 10468, 2045, 2015, 1037, 2155, 2073, 103...",0
4,"[101, 9004, 3334, 4717, 17580, 2293, 1999, 199...",1
...,...,...
63287,"[101, 1045, 2245, 2023, 3185, 2106, 1037, 2091...",1
63288,"[101, 2919, 5436, 2919, 7982, 2919, 3772, 1004...",0
63289,"[101, 1045, 2572, 1037, 3234, 4036, 1999, 2877...",0
63290,"[101, 10047, 2183, 2000, 2031, 2000, 21090, 20...",0


In [ ]:
import re
def preproc(x):
    x = re.sub(r"","", x)
    x = re.sub(r"[^a-zA-Z0-9\s]", "", x)
    x = x.lower()
    return x
df["review"] = df["review"].apply(preproc)
df["sentiment"].replace({"positive" : 1, "negative" : 0}, inplace=True)

<ipython-input-31-39b8090351b2>:8: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["sentiment"].replace({"positive" : 1, "negative" : 0}, inplace=True)
<ipython-input-31-39b8090351b2>:8: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df["sentiment"].replace({"positive" : 1, "negative" : 0}, inplace=

In [ ]:
def tok(sentence):
    return tokenizer.encode(sentence, add_special_tokens=True)
df["review"] = df["review"].apply(tok)
df

,review,sentiment
0,"[101, 2028, 1997, 1996, 2060, 15814, 2038, 385...",1
1,"[101, 1037, 6919, 2210, 2537, 7987, 7987, 1996...",1
2,"[101, 1045, 2245, 2023, 2001, 1037, 6919, 2126...",1
3,"[101, 10468, 2045, 2015, 1037, 2155, 2073, 103...",0
4,"[101, 9004, 3334, 4717, 17580, 2293, 1999, 199...",1
...,...,...
49995,"[101, 1045, 2245, 2023, 3185, 2106, 1037, 2091...",1
49996,"[101, 2919, 5436, 2919, 7982, 2919, 3772, 1004...",0
49997,"[101, 1045, 2572, 1037, 3234, 4036, 1999, 2877...",0
49998,"[101, 10047, 2183, 2000, 2031, 2000, 21090, 20...",0


In [ ]:
max_len = 200
l = []
for i in range(len(df)):
    if len(df.iloc[i, 0]) > max_len:
        l.append([df.iloc[i, 0][:max_len], df.iloc[i, 1]])
        x = df.iloc[i, 0][max_len:]
        df.at[i, df.columns[0]] = df.iloc[i, 0][:max_len]
        for j in range(len(x) // 200):
            if j == len(x) // 2:
                l.append([x[200 * (j + 1) :], df.iloc[i, 1]])
            else:
                l.append([x[200 * (j + 1) : 200 * (j + 2)], df.iloc[i, 1]])
    else:
        l.append([df.iloc[i, 0], df.iloc[i, 1]])

In [ ]:
df1 = pd.DataFrame(l, columns=["review", "sentiment"])
df1


,review,sentiment
0,"[101, 2028, 1997, 1996, 2060, 15814, 2038, 385...",1
1,"[101, 1037, 6919, 2210, 2537, 7987, 7987, 1996...",1
2,"[101, 1045, 2245, 2023, 2001, 1037, 6919, 2126...",1
3,"[101, 10468, 2045, 2015, 1037, 2155, 2073, 103...",0
4,"[101, 9004, 3334, 4717, 17580, 2293, 1999, 199...",1
...,...,...
63287,"[101, 1045, 2245, 2023, 3185, 2106, 1037, 2091...",1
63288,"[101, 2919, 5436, 2919, 7982, 2919, 3772, 1004...",0
63289,"[101, 1045, 2572, 1037, 3234, 4036, 1999, 2877...",0
63290,"[101, 10047, 2183, 2000, 2031, 2000, 21090, 20...",0


In [6]:
def pad(word):
    if len(word) < 200:
        word.extend([0] * (200 - len(word)))
    return word
df1["review"] = df1["review"].apply(pad)
df1

,review,sentiment
0,"[101, 2028, 1997, 1996, 2060, 15814, 2038, 385...",1
1,"[101, 1037, 6919, 2210, 2537, 7987, 7987, 1996...",1
2,"[101, 1045, 2245, 2023, 2001, 1037, 6919, 2126...",1
3,"[101, 10468, 2045, 2015, 1037, 2155, 2073, 103...",0
4,"[101, 9004, 3334, 4717, 17580, 2293, 1999, 199...",1
...,...,...
63287,"[101, 1045, 2245, 2023, 3185, 2106, 1037, 2091...",1
63288,"[101, 2919, 5436, 2919, 7982, 2919, 3772, 1004...",0
63289,"[101, 1045, 2572, 1037, 3234, 4036, 1999, 2877...",0
63290,"[101, 10047, 2183, 2000, 2031, 2000, 21090, 20...",0


In [8]:
df1.to_csv("data.csv", index=False)

In [7]:
(df1["review"].apply(len))

,review
0,200
1,200
2,200
3,200
4,200
...,...
63287,200
63288,200
63289,200
63290,200
